In [5]:
import os, sys
import glob
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py as h5
from tqdm import tqdm, trange

In [6]:
import h5py
from pyproj import Transformer
from scipy.spatial import cKDTree

In [8]:
# from makegif import make_gif
import cartopy
from herbie import Herbie
# from toolbox import EasyMap, pc
# yrb_wbd = np.loadtxt('/global/homes/l/lizh142/hrrr/xyz_csv/yrb_wbd.xyz')
# naches_wbd = np.loadtxt('/global/homes/l/lizh142/hrrr/xyz_csv/naches_wbd.xyz')

In [9]:
def read_daymet_h5(filename):
    data = {}
    with h5py.File(filename, 'r') as f:
        for k, v in f.items():
            try:
                data[k] = v[:]
            except TypeError:
                data_t = {}
                for tk, tv in v.items():
                    data_t[tk] = tv[:]
                data[k] = data_t
    return data

In [10]:
def write_daymet_h5(filename, data):
    with h5py.File(filename, 'w') as f:
        for k, v in data.items():
            try:
                f.create_dataset(k, data=v)
            except TypeError:
                g = f.create_group(k)
                for tk, tv in v.items():
                    g.create_dataset(tk, data=tv)

In [11]:
def check_keys(data):
    keys = data.keys()
    for key in keys:
        print(key, type(data[key]))
        if isinstance(data[key], dict):
            _keys = data[key].keys()
            for _key in _keys:
                if int(_key) > 5:
                    break
                print('\t', _key, ': type is', type(data[key][_key]))

In [12]:
plt.style.use('ggplot')
plt.rcParams['figure.dpi'] = 100
# plt.rcParams['figure.figsize'] = (10,4)
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10

# Get keys and units

In [15]:
run = '2024-07-14-142626' # '2024-07-08-110357' '2024-07-14-142626' ignition:'2024-08-27-132724' post-fire:'2024-08-27-150810'
                          # fire: '2024-11-20-210107' no fire: '2024-11-20-220109'
path = f'/pscratch/sd/l/lizh142/elm/ELM_MOSART_CONUS.{run}/run/'
path = f'/compass/ber200003/zhi/elm/ELM_MOSART_CONUS.{run}/run/'
years = np.arange(2021, 2024)

In [16]:
f = path+f'ELM_MOSART_CONUS.{run}.elm.h0.{years[-1]}-01-01-00000.nc'
# f = path+f'ELM_MOSART_CONUS.{run}.elm.h0.2021-08-05-00000.nc'
data = xr.open_dataset(f)
data

<xarray.Dataset> Size: 31GB
Dimensions:                   (levgrnd: 15, levlak: 10, levdcmp: 15,
                               levsoi: 10, time: 365, hist_interval: 2,
                               lon: 144, lat: 96, ltype: 9, natpft: 17)
Coordinates:
  * levgrnd                   (levgrnd) float32 60B 0.007101 0.02792 ... 35.18
  * levlak                    (levlak) float32 40B 0.05 0.6 2.1 ... 34.33 44.78
  * levdcmp                   (levdcmp) float32 60B 0.007101 0.02792 ... 35.18
  * levsoi                    (levsoi) float32 40B 0.007101 0.02792 ... 2.865
  * time                      (time) object 3kB 2023-01-01 00:00:00 ... 2023-...
  * lon                       (lon) float32 576B 238.4 238.4 ... 239.6 239.6
  * lat                       (lat) float32 384B 46.4 46.41 46.42 ... 47.19 47.2
Dimensions without coordinates: hist_interval, ltype, natpft
Data variables: (12/553)
    mcdate                    (time) int32 1kB ...
    mcsec                     (time) int32 1kB ...
    mdcur                     (time) int32 1kB ...
    mscur                     (time) int32 1kB ...
    nstep                     (time) int32 1kB ...
    time_bounds               (time, hist_interval) object 6kB ...
    ...                        ...
    leaf_npimbalance          (time, lat, lon) float32 20MB ...
    nlim_m                    (time, lat, lon) float32 20MB ...
    o2_decomp_depth_unsat     (time, levgrnd, lat, lon) float32 303MB ...
    plim_m                    (time, lat, lon) float32 20MB ...
    water_scalar              (time, lat, lon) float32 20MB ...
    wlim_m                    (time, lat, lon) float32 20MB ...
Attributes: (12/29)
    title:                                     ELM History file information
    source:                                    E3SM Land Model
    source_id:                                 35b65cea91
    product:                                   model-output
    realm:                                     land
    case:                                      ELM_MOSART_CONUS.2024-07-14-14...
    ...                                        ...
    ltype_wetland:                             6
    ltype_urban_tbd:                           7
    ltype_urban_hd:                            8
    ltype_urban_md:                            9
    Time_constant_3Dvars_filename:             ./ELM_MOSART_CONUS.2024-07-14-...
    Time_constant_3Dvars:                      ZSOI:DZSOI:WATSAT:SUCSAT:BSW:H...

In [17]:
keys, names, units = [], [], []
for key in list(data.keys()):
    try:
        keys.append(key)
    except:
        keys.append('')
    try:
        names.append(data[key].long_name)
    except:
        names.append('')
    try:
        units.append(data[key].units)
    except:
        units.append('')
df = pd.DataFrame(data={'keys': keys, 'names': names, 'units': units})
# df.to_csv('all_ELM_vars.csv')
df

,keys,names,units
0,mcdate,current date (YYYYMMDD),
1,mcsec,current seconds of current date,s
2,mdcur,current day (from base day),
3,mscur,current seconds of current day,
4,nstep,time step,
...,...,...,...
548,nlim_m,runmean N limitation factor,
549,o2_decomp_depth_unsat,o2_decomp_depth_unsat,mol/m3/2
550,plim_m,runmean P limitation factor,
551,water_scalar,water limitation factor for plant dynamic allo...,


In [18]:
# search keyword
search_keyword = 'snow'
idx = []
for i in range(len(df)):
    if search_keyword in df['keys'][i] or search_keyword in df['names'][i] or search_keyword in df['units'][i]:
        idx.append(i)
df.iloc[idx, :]

,keys,names,units
83,EFLX_GRND_LAKE,"net heat flux into lake/snow surface, excludin...",W/m^2
90,ERRH2OSNO,imbalance in snow depth (liquid water),mm
103,FGR,heat flux into soil/snow including snow melt a...,W/m^2
105,FGR_R,Rural heat flux into soil/snow including snow ...,W/m^2
106,FGR_U,Urban heat flux into soil/snow including snow ...,W/m^2
151,FSM,snow melt heat flux,W/m^2
152,FSM_R,Rural snow melt heat flux,W/m^2
153,FSM_U,Urban snow melt heat flux,W/m^2
154,FSNO,fraction of ground covered by snow,1
155,FSNO_EFF,effective fraction of ground covered by snow,1


# Plot examples (optional)

## Plot ET

In [ ]:
_keys = ['QVEGT', 'QSOIL', 'QVEGE']

In [ ]:
_names, _units = [], []
for _key in _keys:
    for i, key in enumerate(keys):
        if key == _key:
            _units.append(units[i].replace('s', 'd'))
            _names.append(names[i])
pd.DataFrame(data={'keys': _keys, 'names': _names, 'units': _units})

In [ ]:
for year in years:
    f = path+f'ELM_MOSART_CONUS.{run}.elm.h0.{year}-01-01-00000.nc'
    data = xr.open_dataset(f)
    lon, lat = data.coords['lon'].values-360, data.coords['lat'].values
    X, Y = np.meshgrid(lon, lat)
    
    for t in tqdm(range(365)):
        fig, axs = plt.subplots(2, 3, figsize=(15, 10), subplot_kw={'projection': cartopy.crs.LambertConformal()})
        et = 0
        for i, ax in enumerate(axs.flat):
            if i < 3:
                EasyMap("10m", ax=ax, crs=cartopy.crs.LambertConformal()).STATES().OCEAN().LAND().COUNTIES().LAKES()
                ax.set_extent([lon.min(), lon.max(), lat.min(), lat.max()], crs=cartopy.crs.PlateCarree())
                ax.add_feature(cartopy.feature.RIVERS)
                et += data[_keys[i]].values[t]*86400
                art = ax.pcolormesh(X, Y, data[_keys[i]].values[t]*86400, cmap='Spectral_r', transform=cartopy.crs.PlateCarree(), vmin=0, vmax=1)
                fig.colorbar(art, label=_units[0], shrink=0.5, extend='max', orientation='vertical', pad=0.02)
                ax.set_title(_keys[i]+'\n'+_names[i]+'\n['+_units[i]+']')
                ax.plot(naches_wbd[:,0]+360, naches_wbd[:,1], 'k', lw=2, transform=pc)
            elif i == 3:
                ax.axis('off')
                continue
            elif i == 4:
                EasyMap("10m", ax=ax, crs=cartopy.crs.LambertConformal()).STATES().OCEAN().LAND().COUNTIES().LAKES()
                ax.set_extent([lon.min(), lon.max(), lat.min(), lat.max()], crs=cartopy.crs.PlateCarree())
                ax.add_feature(cartopy.feature.RIVERS)
                art = ax.pcolormesh(X, Y, et, cmap='Spectral_r', transform=cartopy.crs.PlateCarree(), vmin=0, vmax=1)
                fig.colorbar(art, label=_units[0], shrink=0.5, extend='max', orientation='vertical', pad=0.02)
                ax.set_title(f'Total ET\n{year}, Day {t+1}', fontsize=24)
                ax.plot(naches_wbd[:,0]+360, naches_wbd[:,1], 'k', lw=2, transform=pc)
            elif i == 5:
                EasyMap("10m", ax=ax, crs=cartopy.crs.LambertConformal()).STATES().OCEAN().LAND().COUNTIES().LAKES()
                ax.set_extent([lon.min(), lon.max(), lat.min(), lat.max()], crs=cartopy.crs.PlateCarree())
                ax.add_feature(cartopy.feature.RIVERS)
                art = ax.pcolormesh(X, Y, data['EFLX_LH_TOT'].values[t]*0.0345, cmap='Spectral_r', transform=cartopy.crs.PlateCarree(), vmin=0, vmax=1)
                fig.colorbar(art, label=_units[0], shrink=0.5, extend='max', orientation='vertical', pad=0.02)
                ax.set_title(f'Total ET from LH\n{year}, Day {t+1}', fontsize=24)
                ax.plot(naches_wbd[:,0]+360, naches_wbd[:,1], 'k', lw=2, transform=pc)
        plt.tight_layout()
        plt.savefig(f'./figs/{year}_{str(t).zfill(4)}.jpg')
        plt.close()

In [ ]:
make_gif('./figs/', fps=10, gif_fname=f'{year}.gif')

In [ ]:
fluxes = True # True False

In [ ]:
if fluxes:
    _keys = [#'CWDC_TO_LITR2C', 'CWDC_TO_LITR3C', 
     'LITR1C_TO_SOIL1C', 'LITR2C_TO_SOIL2C', 'LITR3C_TO_SOIL3C', 
     'SOIL1C_TO_SOIL2C', 'SOIL2C_TO_SOIL3C', 'SOIL3C_TO_SOIL4C']
else:
    _keys = ['CWDC', 'LITR1C', 'LITR2C', 'LITR3C', 'SOIL1C', 'SOIL2C', 'SOIL3C', 'SOIL4C']

In [ ]:
_names, _units = [], []
for _key in _keys:
    for i, key in enumerate(keys):
        if key == _key:
            _units.append(units[i])#.replace('/s', '/d'))#.replace('gC', 'mgC')
            _names.append(names[i])
pd.DataFrame(data={'keys': _keys, 'names': _names, 'units': _units})

## Plot soil moisture

In [ ]:
# plot soil moisture
lon, lat = data.coords['lon'].values-360, data.coords['lat'].values
X, Y = np.meshgrid(lon, lat)

fig, axs = plt.subplots(5, 3, figsize=(12,12), subplot_kw={'projection': cartopy.crs.LambertConformal()})
for i, ax in enumerate(axs.flat):
    EasyMap("10m", ax=ax, crs=cartopy.crs.LambertConformal()).STATES().OCEAN().LAND().COUNTIES().LAKES()
    ax.set_extent([lon.min(), lon.max(), lat.min(), lat.max()], crs=cartopy.crs.PlateCarree())
    ax.add_feature(cartopy.feature.RIVERS)
    art = ax.pcolormesh(X, Y, data['H2OSOI'].values[60, i, :, :], cmap='Spectral_r', vmin=0, vmax=1, transform=cartopy.crs.PlateCarree())
    fig.colorbar(art, label='[mm3/mm3]', shrink=0.5, extend='max', orientation='vertical', pad=0.02)
    ax.set_title(f'layer {i+1}: {str(np.around(data.levgrnd.values[i], 3))} m')
    ax.plot(naches_wbd[:,0]+360, naches_wbd[:,1], 'k', lw=2, transform=pc)
plt.tight_layout()
plt.show()

In [ ]:
# plot soil moisture in a soil column
plt.plot(data['H2OSOI'].values[160, :, 0, 0], -np.arange(len(data['H2OSOI'].values[60, :, 0, 0])), '-x')
plt.show()

## Get 2mT, rain, snowmelt, soil water content, and _key data

In [ ]:
tsa, rain, snowmelt, theta = np.zeros(len(years)*365), np.zeros(len(years)*365), np.zeros(len(years)*365), np.zeros(len(years)*365)
for year in tqdm(years):
    f = path+f'ELM_MOSART_CONUS.{run}.elm.h0.{year}-01-01-00000.nc'
    data = xr.open_dataset(f)
    for j in range(365):
        tsa[j+365*(year-years[0])] = data['TSA'].values[j, :, :].mean()-273.15 # K -> deg C
        rain[j+365*(year-years[0])] = data['RAIN'].values[j, :, :].mean()*86400 # mm/s -> mm/d
        snowmelt[j+365*(year-years[0])] = data['QSNOMELT'].values[j, :, :].mean()*86400 # mm/s -> mm/d
        theta[j+365*(year-years[0])] = data['H2OSOI'][j, :10, :, :].mean() # mm3/mm3

In [ ]:
_keys

In [ ]:
d = np.zeros((len(years)*365, len(_keys)))
for year in tqdm(years):
    f = path+f'ELM_MOSART_CONUS.{run}.elm.h0.{year}-01-01-00000.nc'
    data = xr.open_dataset(f)
    for j in range(365):
        for i in range(len(_keys)):
            d[j+365*(year-years[0]), i] = data[_keys[i]].values[j, :, :].mean()#*86400#*1000 # gC/m^2/s -> gC/m^2/d

## Plot all _keys data

In [ ]:
_units

In [ ]:
plt.figure(figsize=(20, 4))
for i in range(len(_keys)):
    plt.plot(d[:, i], label=_keys[i])
plt.xticks(np.arange(0, d.shape[0], 365), 
           np.arange(0, d.shape[0], 365)//365+1981, rotation=45)
plt.legend(ncols=4, edgecolor='none', facecolor='none')
plt.xlim(365*35, d.shape[0])
plt.grid(ls='--')
plt.ylabel(_units[0])

plt.gca().twinx().plot(tsa, label='tsa', lw=3, color='navy', alpha=0.3)
plt.ylim(-20, 40)
plt.ylabel('[deg C]')

# plt.gca().twinx().plot(rain, label='rain', lw=3, color='navy', alpha=0.3)
# plt.ylim(-25, 110)
# plt.gca().invert_yaxis()
# plt.ylabel('[mm/d]')

# plt.gca().twinx().plot(snowmelt, label='snowmelt', lw=3, color='navy', alpha=0.3)
# plt.ylim(-25, 110)
# plt.gca().invert_yaxis()
# plt.ylabel('[mm/d]')

plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(20, 4))
plt.plot(theta)
plt.xticks(np.arange(0, d.shape[0], 365), 
           np.arange(0, d.shape[0], 365)//365+1981, rotation=45)
plt.xlim(365*35, d.shape[0])
plt.grid(ls='--')
plt.ylabel('[mm3/mm3]')

plt.gca().twinx().plot(tsa, label='tsa', lw=3, color='navy', alpha=0.3)
plt.ylim(-20, 40)
plt.ylabel('[deg C]')

# plt.gca().twinx().plot(rain, label='rain', lw=3, color='navy', alpha=0.3)
# plt.ylim(-25, 110)
# plt.gca().invert_yaxis()
# plt.ylabel('[mm/d]')

# plt.gca().twinx().plot(snowmelt, label='snowmelt', lw=3, color='navy', alpha=0.3)
# plt.ylim(-25, 110)
# plt.gca().invert_yaxis()
# plt.ylabel('[mm/d]')

plt.grid()
plt.show()

## Typical year

In [ ]:
t_typ, rain_typ, snowmelt_typ, theta_typ, d_typ = 0, 0, 0, 0, 0
for i in range(len(years)):
    t_typ += tsa[i*365:(i+1)*365]
    rain_typ += rain[i*365:(i+1)*365]
    snowmelt_typ += snowmelt[i*365:(i+1)*365]
    theta_typ += theta[i*365:(i+1)*365]
    d_typ += d[i*365:(i+1)*365, :]
t_typ /= len(years)
rain_typ /= len(years)
snowmelt_typ /= len(years)
theta_typ /= len(years)
d_typ /= len(years)

In [ ]:
np.mean(d_typ, axis=1).shape

In [ ]:
plt.figure(figsize=(10, 4))
for i in range(len(_keys)):
    plt.plot(d_typ[:, i], lw=2, label=_keys[i])
plt.plot(np.mean(d_typ, axis=1), lw=3, c='k')
plt.xticks(np.linspace(0, 364, 13), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'Jan'])
plt.legend(ncols=4, edgecolor='none', facecolor='none')
plt.grid(ls='--')
plt.ylabel(_units[0])
# plt.ylim(0, 160)

plt.gca().twinx().plot(t_typ, label='tsa', lw=8, color='navy', alpha=0.3)
plt.ylim(-15, 30)
plt.ylabel('[deg C]')

# plt.gca().twinx().plot(rain_typ, label='rain', lw=3, color='navy', alpha=0.3)
# plt.ylim(-1, 6)
# plt.gca().invert_yaxis()
# plt.ylabel('[mm/d]')

# plt.gca().twinx().plot(snowmelt_typ, label='snowmelt', lw=3, color='navy', alpha=0.3)
# plt.ylim(-1, 6)
# plt.gca().invert_yaxis()
# plt.ylabel('[mm/d]')

plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(theta_typ, lw=2)
plt.xticks(np.linspace(0, 364, 13), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'Jan'])
plt.grid(ls='--')
plt.ylabel(_units[0])
# plt.ylim(0, 160)

plt.gca().twinx().plot(t_typ, label='tsa', lw=8, color='navy', alpha=0.3)
plt.ylim(-8, 23)
# plt.gca().invert_yaxis()
plt.ylabel('[deg C]')

# plt.gca().twinx().plot(rain_typ, label='rain', lw=3, color='navy', alpha=0.3)
# plt.ylim(-1, 6)
# plt.gca().invert_yaxis()
# plt.ylabel('[mm/d]')

# plt.gca().twinx().plot(snowmelt_typ, label='snowmelt', lw=3, color='navy', alpha=0.3)
# plt.ylim(-1, 6)
# plt.gca().invert_yaxis()
# plt.ylabel('[mm/d]')

plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(10, 4))
total_flux = np.sum(d_typ, axis=1)
plt.plot(total_flux, lw=2)
plt.xticks(np.linspace(0, 364, 13), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'Jan'])
plt.grid(ls='--')
plt.ylabel(_units[0])
# plt.ylim(0, 160)

plt.gca().twinx().plot(t_typ, label='tsa', lw=8, color='navy', alpha=0.3)
plt.ylim(-8, 23)
# plt.gca().invert_yaxis()
plt.ylabel('[deg C]')

plt.grid()
plt.show()

## Make daily images and GIFs

In [ ]:
for year in tqdm(years):
    f = path+f'ELM_MOSART_CONUS.{run}.elm.h0.{year}-01-01-00000.nc'
    data = xr.open_dataset(f)
    lon, lat = data.coords['lon'].values-360, data.coords['lat'].values
    X, Y = np.meshgrid(lon, lat)
    
    if year == years[0]:
        keys, names, units = [], [], []
        for key in list(data.keys()):
            try:
                keys.append(key)
            except:
                keys.append('')
            try:
                names.append(data[key].long_name)
            except:
                names.append('')
            try:
                units.append(data[key].units)
            except:
                units.append('')

        _keys = ['CWDC_TO_LITR2C', 'CWDC_TO_LITR3C', '',
         'LITR1C_TO_SOIL1C', 'LITR2C_TO_SOIL2C', 'LITR3C_TO_SOIL3C', 
         'SOIL1C_TO_SOIL2C', 'SOIL2C_TO_SOIL3C', 'SOIL3C_TO_SOIL4C']
        _names, _units = [], []
        for j, _key in enumerate(_keys):
            if j == 2:
                _units.append('')
                _names.append('')
            for i, key in enumerate(keys):
                if key == _key:
                    _units.append(units[i].replace('gC', 'mgC').replace('/s', '/d'))
                    _names.append(names[i])

    for t in tqdm(range(365)):
        fig, axs = plt.subplots(3, 3, figsize=(15, 15), subplot_kw={'projection': cartopy.crs.LambertConformal()})
        for i, ax in enumerate(axs.flat):
            if i == 2:
                ax.set_title(f'{year}\nDay {t+1}', fontsize=36)
                ax.axis('off')
                continue
            EasyMap("10m", ax=ax, crs=cartopy.crs.LambertConformal()).STATES().OCEAN().LAND().COUNTIES().LAKES()
            ax.set_extent([lon.min(), lon.max(), lat.min(), lat.max()], crs=cartopy.crs.PlateCarree())
            ax.add_feature(cartopy.feature.RIVERS)
            art = ax.pcolormesh(X, Y, data[_keys[i]].values[t]*1000*86400, cmap='Spectral_r', vmin=0, vmax=300, transform=cartopy.crs.PlateCarree())
            fig.colorbar(art, label='[mgC/m^2/d]', shrink=0.5, extend='max', orientation='vertical', pad=0.02)
            ax.set_title('\n\n\n'+_keys[i]+'\n'+_names[i])#+'\n['+_units[i]+']')
            ax.plot(naches_wbd[:,0]+360, naches_wbd[:,1], 'k', lw=2, transform=pc)
        plt.tight_layout()
        plt.savefig(f'./figs/{year}_{str(t).zfill(4)}.jpg')
        plt.close()
    
    make_gif(fps=10, gif_fname=f'{year}.gif')

In [ ]:
make_gif('./figs/', fps=10, gif_fname=f'{year}.gif')

# Export hdf5

## Find x and y

In [19]:
lon, lat = data.coords['lon'].values-360, data.coords['lat'].values
X, Y = np.meshgrid(lon, lat)

In [20]:
lonlat = np.zeros((len(X.flatten()), 2))
lonlat[:, 0], lonlat[:, 1] = X.flatten(),  Y.flatten()

In [21]:
proj_lcc = "+proj=lcc +lat_1=25 +lat_2=60 +lat_0=42.5 +lon_0=-100 +x_0=0 +y_0=0 +datum=WGS84" # daymet crs
proj_wgs84 = "epsg:4326" # latlong
lonlat_to_daymet = np.array(Transformer.from_crs(proj_wgs84, proj_lcc).transform(lonlat[:, 1], lonlat[:, 0]))

In [22]:
# a, b = np.reshape(lonlat_to_daymet[0], X.shape), np.reshape(lonlat_to_daymet[1], Y.shape)
# area = np.zeros((a.shape[0]-1, a.shape[1]-1))
# for i in range(a.shape[0]-1):
#     for j in range(a.shape[1]-1):
#         area[i, j] = (a[i, j] - a[i+1, j+1]) * (b[i, j] - b[i+1, j+1])
# area.mean()

In [23]:
xv, yv = np.meshgrid(np.linspace(lonlat_to_daymet[0].min(), lonlat_to_daymet[0].max(), 120), 
                     np.linspace(lonlat_to_daymet[1].min(), lonlat_to_daymet[1].max(), 120))

In [24]:
# plt.scatter(lonlat_to_daymet[0], lonlat_to_daymet[1], s=0.5)
# plt.scatter(xv, yv, s=0.5)
# plt.axis('equal')
# plt.show()

In [25]:
def idw_interpolation(xy_target, xy_source, values, power=2):
    tree = cKDTree(xy_source)
    distances, indices = tree.query(xy_target, k=4)
    weights = 1 / (distances ** power)
    weights /= weights.sum(axis=1, keepdims=True)
    interpolated_values = np.sum(values[indices] * weights, axis=1)
    return interpolated_values

In [26]:
xy_source = np.zeros((len(lonlat_to_daymet[0]), 2))
xy_source[:, 0], xy_source[:, 1] = lonlat_to_daymet[0], lonlat_to_daymet[1]
xy_target = np.zeros((len(xv.flatten()), 2))
xy_target[:, 0], xy_target[:, 1] = xv.flatten(), yv.flatten()

## Find ELM data

In [27]:
fluxes = True # True False

In [29]:
pre_fire = True # True False

In [30]:
if fluxes:
    _keys = [#'CWDC_TO_LITR2C', 'CWDC_TO_LITR3C', 
     'LITR1C_TO_SOIL1C', 'LITR2C_TO_SOIL2C', 'LITR3C_TO_SOIL3C', 
     'SOIL1C_TO_SOIL2C', 'SOIL2C_TO_SOIL3C', 'SOIL3C_TO_SOIL4C']
else:
    _keys = ['CWDC', 'LITR1C', 'LITR2C', 'LITR3C', 'SOIL1C', 'SOIL2C', 'SOIL3C', 'SOIL4C']

In [31]:
if fluxes:
    elm_data = {
        "DOC production latlon [molC m^-2 s^-1]": {},
        "DOC production [molC m^-2 s^-1]": {},
        "time [s]": (np.arange(len(years)*365)+365*(years[0]-1980))*86400,
        "x [m]": xv[0,:],
        "y [m]": yv[:,0],
    }
else:
    elm_data = {
        "time [s]": (np.arange(len(years)*365)+365*(years[0]-1980))*86400,
        "x [m]": xv[0,:],
        "y [m]": yv[:,0],
    }
    for _key in _keys:
        elm_data[_key+' [molC m^-2]'] = {}
        elm_data[_key+' latlon [molC m^-2]'] = {}

In [32]:
if pre_fire:
    for year in tqdm(years):
        f = path+f'ELM_MOSART_CONUS.{run}.elm.h0.{year}-01-01-00000.nc'
        data = xr.open_dataset(f)
        for day in range(365):
            label = str(day+365*(year-years[0]))
            if fluxes:
                f_DOM, fdom = 0.01, '001'
                elm_data['DOC production latlon [molC m^-2 s^-1]'][label] = 0
                theta = data['H2OSOI'].values[day, :10, :, :].mean(axis=0)
                theta[np.isnan(theta)] = 0 
                assert(len(np.unique(np.isnan(theta))) == 1)
                for i, _key in enumerate(_keys):
                    assert(len(np.unique(np.isnan(elm_data['DOC production latlon [molC m^-2 s^-1]'][label]))) == 1)
                    elm_data['DOC production latlon [molC m^-2 s^-1]'][label] += data[_key].values[day, :, :]*f_DOM*theta/12#*area.mean()
                elm_data['DOC production latlon [molC m^-2 s^-1]'][label] /= len(_keys)
                values = elm_data['DOC production latlon [molC m^-2 s^-1]'][label].flatten()
                interpolated_values = idw_interpolation(xy_target, xy_source, values)
                elm_data['DOC production [molC m^-2 s^-1]'][label] = interpolated_values.reshape(120, 120)
            else:
                for i, _key in enumerate(_keys):
                    elm_data[f'{_key} latlon [molC m^-2]'][label] = data[_key].values[day, :, :]
                    values = elm_data[f'{_key} latlon [molC m^-2]'][label].flatten()
                    interpolated_values = idw_interpolation(xy_target, xy_source, values)
                    elm_data[f'{_key} [molC m^-2]'][label] = interpolated_values.reshape(120, 120)

100%|██████████| 3/3 [00:32<00:00, 10.87s/it]


In [33]:
if not pre_fire:
    run = '2024-11-20-220109' # post-fire:'2024-08-27-150810'
                              # fire: '2024-11-20-210107' no fire: '2024-11-20-220109'
    path = f'/pscratch/sd/l/lizh142/elm/ELM_MOSART_CONUS.{run}/run/'
    fl = sorted(glob.glob(path+f'ELM_MOSART_CONUS.{run}.elm.h0.*.nc'))
    for i, f in enumerate(tqdm(fl)):
        data = xr.open_dataset(f)
        label = str(i)
        if fluxes:
            f_DOM, fdom = 0.01, '001'
            elm_data['DOC production latlon [molC m^-2 s^-1]'][label] = 0
            theta = data['H2OSOI'].values[0, :10, :, :].mean(axis=0)
            theta[np.isnan(theta)] = 0 
            assert(len(np.unique(np.isnan(theta))) == 1)
            for i, _key in enumerate(_keys):
                assert(len(np.unique(np.isnan(elm_data['DOC production latlon [molC m^-2 s^-1]'][label]))) == 1)
                elm_data['DOC production latlon [molC m^-2 s^-1]'][label] += data[_key].values[0, :, :]*f_DOM*theta/12#*area.mean()
            elm_data['DOC production latlon [molC m^-2 s^-1]'][label] /= len(_keys)
            values = elm_data['DOC production latlon [molC m^-2 s^-1]'][label].flatten()
            interpolated_values = idw_interpolation(xy_target, xy_source, values)
            elm_data['DOC production [molC m^-2 s^-1]'][label] = interpolated_values.reshape(120, 120)
        else:
            for i, _key in enumerate(_keys):
                elm_data[f'{_key} latlon [molC m^-2]'][label] = data[_key].values[0, :, :]
                values = elm_data[f'{_key} latlon [molC m^-2]'][label].flatten()
                interpolated_values = idw_interpolation(xy_target, xy_source, values)
                elm_data[f'{_key} [molC m^-2]'][label] = interpolated_values.reshape(120, 120)
    elm_data["time [s]"] = (np.arange(len(fl)) + 15180)*86400

In [36]:
# loop year: fluxes
if pre_fire:
    if fluxes:
        write_daymet_h5(f'/home/lizh142/Naches_DOC_{years[0]}_{years[-1]}_fdom{fdom}_{run}.h5', elm_data)
    else:
        write_daymet_h5(f'/home/lizh142/Naches_Cpools_{years[0]}_{years[-1]}_{run}.h5', elm_data)
else:
    if fluxes:
        write_daymet_h5(f'/home/lizh142/Naches_DOC_{years[0]}_{years[-1]}_fdom{fdom}_postfire_{run}.h5', elm_data)
    else:
        write_daymet_h5(f'/home/lizh142/Naches_Cpools_{years[0]}_{years[-1]}_postfire_{run}.h5', elm_data)

In [37]:
# typical year

In [38]:
typ = np.zeros((365, 
                elm_data['DOC production [molC m^-2 s^-1]']['0'].shape[0], 
                elm_data['DOC production [molC m^-2 s^-1]']['0'].shape[1]))
typ.shape

(365, 120, 120)

In [39]:
for day in trange(365):
    avg = 0
    for year in years:
        label = str(day+365*(year-years[0]))
        avg += elm_data['DOC production [molC m^-2 s^-1]'][label]
    typ[day, :, :] = avg/len(years)

100%|██████████| 365/365 [00:00<00:00, 11135.91it/s]


In [40]:
elm_data_typ = {
    "DOC production [molC m^-2 s^-1]": {},
    "time [s]": (np.arange(len(years)*365)+365*(years[0]-1980))*86400,
    "x [m]": xv[0,:],
    "y [m]": yv[:,0],
}

In [41]:
for year in tqdm(years):
    for day in range(365):
        label = str(day+365*(year-years[0]))
        elm_data_typ['DOC production [molC m^-2 s^-1]'][label] = typ[day, :, :]

100%|██████████| 3/3 [00:00<00:00, 1703.39it/s]


In [42]:
write_daymet_h5(f'/home/lizh142/Naches_DOC_typical_{years[0]}_{years[-1]}_fdom{fdom}.h5', elm_data_typ)